 # Wrangling Data With DuckDB

In [1]:
import duckdb

In [2]:
data = duckdb.sql("select * from 'bea_gdp_virginia_2018-current.csv'")

In [ ]:
data

In [ ]:
duckdb.sql("describe data")

In [ ]:
duckdb.sql("summarize data")

In [6]:
pd_df = data.to_df()

In [7]:
import pandas as pd

In [ ]:
pd_df.describe(include="all").transpose()

In [9]:
data.to_parquet('output.parquet')

In [10]:
duckdb.sql("INSTALL sqlite; LOAD sqlite;")



In [11]:
duckdb.sql("Create table if not exists benchmarks as select * from sqlite_scan('../benchmark_results.db','benchmark_results')")


In [ ]:
duckdb.sql("SELECT * FROM benchmarks").to_df().describe(include="all").transpose()

In [13]:

df = duckdb.sql(""" 
FROM benchmarks
SELECT 
tool
, file_size
, regexp_replace(operation, '\d+$', '') as operation      -- remove trailing integers
, execution_time

""").to_df()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Create the lmplot with legend positioned at the upper middle left
g = sns.lmplot(x='file_size', y='execution_time', data=df, hue='tool', 
               height=6, aspect=10/6,
               legend_out=False)  # Keep legend inside the plot

# Get the figure and axes from the FacetGrid
fig = g.fig
ax = g.axes[0, 0]  # The first (and only) subplot

# Set x-axis to log scale
ax.set_xscale('log')

# Add title and labels
ax.set_title('Execution Time Distribution by Tool', fontsize=16)
ax.set_xlabel('File Size (rows)', fontsize=14)
ax.set_ylabel('Execution Time (seconds)', fontsize=14)

# Add grid for better readability
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Update the legend position
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='upper left', bbox_to_anchor=(0.6, .65))

# Improve aesthetics
sns.despine(left=False, bottom=False, ax=ax)
plt.tight_layout()

# Show the plot
plt.show()

In [17]:

source_data_url = "https://data.virginia.gov/dataset/3c786292-3dd5-4327-adad-79332a31683c/resource/59a6ed5f-11ec-49a2-b394-bbfd3e90fe72/download/eva_procurement_data_2025.csv"
target_file_path = "eVA_procurement_data_2025.parquet"
# 865 MB.

In [14]:
#%%

duckdb.sql("install httpfs; load httpfs;")

In [ ]:
duckdb.from_csv_auto(data_url).to_parquet(target_file_path)